# 1)-Importing key Modules



In [0]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [2]:
! pip install dynet

     |████████████████████████████████| 28.1MB 83kB/s 


In [0]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

In [4]:
! pip install version_information

  Created wheel for version-information: filename=version_information-1.0.3-cp36-none-any.whl size=3880 sha256=716c49de3710bea28124cdf8c62aa394c5f200212a7df93f639ff1353ba1aee9
  Stored in directory: /root/.cache/pip/wheels/1f/4c/b3/1976ac11dbd802723b564de1acaa453a72c36c95827e576321
Successfully built version-information


In [5]:
# first install: pip install version_information
%reload_ext version_information
%version_information pandas,dynet,numpy

Software versions
Python 3.6.8 64bit [GCC 8.3.0]
IPython 5.5.0
OS Linux 4.14.137+ x86_64 with Ubuntu 18.04 bionic
pandas 0.25.3
dynet 2.0.3
numpy 1.17.3
Thu Nov 07 21:49:22 2019 UTC

# 2)- Loading Data

In [0]:
# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

In [0]:
# Read in the data
train = list(read_dataset("train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("test.txt"))

In [8]:
print(len(w2i))
print(len(dev))

18648
2210


In [0]:
nwords = len(w2i)
ntags = len(t2i)

In [10]:
train[0][1]

0

In [0]:
# Start DyNet and define trainer
model = dy.ParameterCollection()
trainer = dy.AdamTrainer(model)

# 3)- Define Model

Using embedding

In [0]:
EMB_SIZE = 64
W_emb = model.add_lookup_parameters((nwords, EMB_SIZE)) # Word embeddings
W_sm = model.add_parameters((ntags, EMB_SIZE))          # Softmax weights
b_sm = model.add_parameters((ntags))                      # Softmax bias

In [0]:
# A function to calculate scores for one value
def calc_scores(words):
  dy.renew_cg()
  cbow = dy.esum([dy.lookup(W_emb, x) for x in words])
  return W_sm * cbow + b_sm

# 4)- Train the Model

In [14]:
for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform testing
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=1.4709, time=0.34s
iter 0: test acc=0.4086
iter 1: train loss/sent=1.0821, time=0.33s
iter 1: test acc=0.4262
iter 2: train loss/sent=0.7704, time=0.36s
iter 2: test acc=0.4081
iter 3: train loss/sent=0.5289, time=0.38s
iter 3: test acc=0.4045
iter 4: train loss/sent=0.3641, time=0.35s
iter 4: test acc=0.3783
iter 5: train loss/sent=0.2509, time=0.32s
iter 5: test acc=0.3937
iter 6: train loss/sent=0.1760, time=0.36s
iter 6: test acc=0.3810
iter 7: train loss/sent=0.1292, time=0.37s
iter 7: test acc=0.3747
iter 8: train loss/sent=0.0953, time=0.37s
iter 8: test acc=0.3991
iter 9: train loss/sent=0.0691, time=0.37s
iter 9: test acc=0.3846
iter 10: train loss/sent=0.0588, time=0.37s
iter 10: test acc=0.3756
iter 11: train loss/sent=0.0455, time=0.36s
iter 11: test acc=0.3769
iter 12: train loss/sent=0.0368, time=0.36s
iter 12: test acc=0.3724
iter 13: train loss/sent=0.0271, time=0.36s
iter 13: test acc=0.3738
iter 14: train loss/sent=0.0294, time=0.36s
iter 14: t